In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve,roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report

In [ ]:
data=pd.read_csv('database_IND.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isna().sum()

The Null vlues in 'other_fuel1','other_fuel2','other_fuel3','commissioning_year','wepp_id',
'year_of_capacity_data','generation_gwh_2013','generation_gwh_2014','generation_gwh_2015','generation_gwh_2016',
'generation_gwh_2017','generation_gwh_2018','generation_gwh_2019','generation_data_source','estimated_generation_gwh' are too much

we cannot impute them all.

And some columns are irrelevant to model building as they dont provide any real data. they have to be deleted

To be deleted - country, country_long, name, source, url for not offering any real numerical or categorically useful data.
Also wepp_id, generation_gwh_2013, estimated_generation_gwh, other_fuel2, other_fuel3 as theyre fully blank features

In [ ]:
data=data.drop(columns=['country','country_long','name','source','url','wepp_id','generation_gwh_2013','estimated_generation_gwh','other_fuel2','other_fuel3','generation_gwh_2019'],axis=1)

In [ ]:
data.shape

In [ ]:
data.isna().sum()

In [ ]:
data['latitude']=data['latitude'].fillna(0)
data['longitude']=data['longitude'].fillna(0)
data['other_fuel1']=data['other_fuel1'].fillna('a')
data['commissioning_year']=data['commissioning_year'].fillna(0)
data['owner']=data['owner'].fillna('a')
data['geolocation_source']=data['geolocation_source'].fillna('a')
data['year_of_capacity_data']=data['year_of_capacity_data'].fillna(0)
data['generation_gwh_2014']=data['generation_gwh_2014'].fillna(0)
data['generation_gwh_2015']=data['generation_gwh_2015'].fillna(0)
data['generation_gwh_2016']=data['generation_gwh_2016'].fillna(0)
data['generation_gwh_2017']=data['generation_gwh_2017'].fillna(0)
data['generation_gwh_2018']=data['generation_gwh_2018'].fillna(0)
data['generation_data_source']=data['generation_data_source'].fillna('a')

In [ ]:
data.isna().sum()

In [ ]:
data.info()

In [ ]:
lab_enc=LabelEncoder()

In [ ]:
data.rename(columns={'other_fuel1':'otherfuel'}, inplace= True)

In [ ]:
df=lab_enc.fit_transform(data['gppd_idnr'])
pd.Series(df)
data['gppd_idnr']=df
df=lab_enc.fit_transform(data['primary_fuel'])
pd.Series(df)
data['primary_fuel']=df
df=lab_enc.fit_transform(data['other_fuel1'])
pd.Series(df)
data['other_fuel1']=df
df=lab_enc.fit_transform(data['owner'])
pd.Series(df)
data['owner']=df
df=lab_enc.fit_transform(data['geolocation_source'])
pd.Series(df)
data['geolocation_source']=df
df=lab_enc.fit_transform(data['generation_data_source'])
pd.Series(df)
data['generation_data_source']=df

In [ ]:
data

In [ ]:
plt.figure(figsize=(20,20),facecolor='white')
p=1
for column in data:
    if p <=16:
        ax=plt.subplot(4,4,p)
        sns.distplot(data[column])
        plt.xlabel(column,fontsize=20)
    p+=1
plt.tight_layout()

# 1) Primary Fuel

In [ ]:
y=data['primary_fuel']
x=data.drop(columns=['primary_fuel'])

In [ ]:
plt.figure(figsize=(17,17),facecolor='white')
plotnumber=1
for column in x:
    if plotnumber<=16:
        ax=plt.subplot(4,4,plotnumber)
        plt.scatter(x[column],y)
        plt.xlabel(column,fontsize=15)
        plt.ylabel('primary_fuel',fontsize=15)
    plotnumber+=1
plt.tight_layout()

In [ ]:
data_corr=data.corr().abs()
plt.figure(figsize=(15,15))
sns.heatmap(data_corr,annot=True,annot_kws={'size':10})
plt.show

In [ ]:
plt.figure(figsize=(15,15),facecolor='white')
plotnumber=1
for column in x:
    if plotnumber<=16:
        ax=plt.subplot(4,4,plotnumber)
        sns.lineplot(x='primary_fuel',y=column,data=data)
        plt.xlabel('primary_fuel',fontsize=10)
        plt.ylabel(column,fontsize=10)
    plotnumber+=1
plt.tight_layout()

In [ ]:
from sklearn.feature_selection import SelectKBest,f_classif

In [ ]:
best_features=SelectKBest(score_func=f_classif,k='all')

fit=best_features.fit(x,y)

data_scores=pd.DataFrame(fit.scores_)

data_columns=pd.DataFrame(x.columns)

feature_scores=pd.concat([data_columns,data_scores],axis=1)

feature_scores.columns=['Feature_Name','Score']

print(feature_scores.nlargest(13,'Score'))

Based on heatmap and select. we can use above features

In [ ]:
y=data['primary_fuel']
x=data[['year_of_capacity_data','gppd_idnr','commissioning_year','other_fuel1','generation_data_source','owner','capacity_mw','generation_gwh_2018','generation_gwh_2017','generation_gwh_2016','generation_gwh_2015','generation_gwh_2014']]

In [ ]:
plt.figure(figsize=(20,20),facecolor='white')
plotno=1
for column in x:
    if plotno<=14:
        ax=plt.subplot(4,4,plotno)
        sns.boxplot(data=x[column])
        plt.xlabel(column,fontsize=12)
    plotno+=1
plt.show()

In [ ]:
scaler=StandardScaler()
x_scaled=scaler.fit_transform(x)
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=0.17,random_state=333)

In [ ]:
def metric_score(clf,x_train,x_test,y_train,y_test,train=True):
    if train:
        y_pred=clf.predict(x_train)
        print('\n----Train result----')
        print(f'Accuracy Score:{accuracy_score(y_train,y_pred)*100:.2f}%')
        
    elif train==False:
        pred=clf.predict(x_test)
        print('\n----Test result----')
        print(f'Accuracy Score:{accuracy_score(y_test,pred)*100:.2f}%')

        print('\n\n Test Classification report \n',classification_report(y_test,pred,digits=2))

In [ ]:
knn=KNeighborsClassifier()
knn.fit(x_train,y_train)

In [ ]:
metric_score(knn,x_train,x_test,y_train,y_test,train=True)
metric_score(knn,x_train,x_test,y_train,y_test,train=False)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression()
log_reg.fit(x_train,y_train)

In [ ]:
metric_score(log_reg,x_train,x_test,y_train,y_test,train=True)
metric_score(log_reg,x_train,x_test,y_train,y_test,train=False)

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
bag_knn=BaggingClassifier(KNeighborsClassifier(n_neighbors=9),
                          n_estimators=6,max_samples=0.75,
                          bootstrap=True,random_state=366,oob_score=True)

In [ ]:
bag_knn.fit(x_train,y_train)

In [ ]:
metric_score(bag_knn,x_train,x_test,y_train,y_test,train=True)
metric_score(bag_knn,x_train,x_test,y_train,y_test,train=False)

Bagging KNN is giving us best results, we move forward with it

In [ ]:
from sklearn.model_selection import KFold,cross_val_score
cross_val_score(bag_knn,x_scaled,y,cv=10)

In [ ]:
pickle.dump(bag_knn,open('Primary Fuel','wb'))

# 2) Capacity

In [ ]:
y=data['capacity_mw']
x=data.drop(columns=['capacity_mw'])

In [ ]:
plt.figure(figsize=(15,15),facecolor='white')
plotnumber=1
for column in x:
    if plotnumber<=16:
        ax=plt.subplot(4,4,plotnumber)
        plt.scatter(x[column],y)
        plt.xlabel(column,fontsize=10)
        plt.ylabel('primary_fuel',fontsize=10)
    plotnumber+=1
plt.tight_layout()

In [ ]:
data_corr=data.corr().abs()
plt.figure(figsize=(15,15))
sns.heatmap(data_corr,annot=True,annot_kws={'size':10})
plt.show

In [ ]:
scaler=StandardScaler()
x_scaled=scaler.fit_transform(x)

In [ ]:
vif=pd.DataFrame()
vif['vif']=[variance_inflation_factor(x_scaled,i) for i in range(x_scaled.shape[1])]
vif['Features']=x.columns
vif

In [ ]:
from sklearn.feature_selection import SelectKBest,f_regression

In [ ]:
best_features=SelectKBest(score_func=f_regression,k='all')

fit=best_features.fit(x,y)

data_scores=pd.DataFrame(fit.scores_)

data_columns=pd.DataFrame(x.columns)

feature_scores=pd.concat([data_columns,data_scores],axis=1)

feature_scores.columns=['Feature_Name','Score']

print(feature_scores.nlargest(11,'Score'))

In [ ]:
y=data['capacity_mw']
x=data[['generation_gwh_2018','generation_gwh_2017','generation_gwh_2016','generation_gwh_2015','generation_gwh_2014','other_fuel1','generation_data_source','year_of_capacity_data','commissioning_year','primary_fuel','gppd_idnr']]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=0.17,random_state=44)

In [ ]:
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(x_train,y_train)

In [ ]:
reg.score(x_train,y_train)

In [ ]:
reg.score(x_test,y_test)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import Lasso,Ridge,RidgeCV,LassoCV

In [ ]:
lasscv=LassoCV(alphas=None,max_iter=300,normalize=True)
lasscv.fit(x_train,y_train)

In [ ]:
alpha=lasscv.alpha_
alpha

In [ ]:
lasso_reg=Lasso(alpha)
lasso_reg.fit(x_train,y_train)

In [ ]:
lasso_reg.score(x_train,y_train)

In [ ]:
lasso_reg.score(x_test,y_test)

In [ ]:
ridgecv=RidgeCV(alphas=np.arange(0.002,0.1,0.02),normalize=True)
ridgecv.fit(x_train,y_train)

In [ ]:
ridgecv.alpha_

In [ ]:
ridge_model=Ridge(alpha=ridgecv.alpha_)
ridge_model.fit(x_train,y_train)

In [ ]:
ridge_model.score(x_test,y_test)

Lasso Reg has the best score, moving forward with that

In [ ]:
y_pred=reg.predict(x_test)

In [ ]:
plt.scatter(y_test,y_pred)
plt.xlabel('Actual capacity')
plt.ylabel('Predicted capacity')
plt.title('Actual vs Model Predicted')
plt.show()

In [ ]:
y_pred=reg.predict(x_test)

In [ ]:
mean_absolute_error(y_test,y_pred)

In [ ]:
mean_squared_error(y_test,y_pred)

In [ ]:
np.sqrt(mean_squared_error(y_test,y_pred))

In [ ]:
pickle.dump(lasso_reg,open('Power Plant capacity','wb'))